# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [0]:
from pyspark.sql import SparkSession
spark  = SparkSession.builder.appName('lrex').getOrCreate()

In [0]:
data = spark.read.csv('/FileStore/tables/cruise_ship_info.csv', inferSchema=True, header= True)
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [0]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="Cruise_line", outputCol="cruise_cast")
indexed = indexer.fit(data).transform(data)
indexed.columns

Out[51]: ['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_cast']

In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors

In [0]:
assembler= VectorAssembler(inputCols=['Age', 'Tonnage', 'passengers','length','cabins','passenger_density', 'cruise_cast'], outputCol= 'features')
output = assembler.transform(indexed)
output.select("features", "crew").show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [0]:
# split it
train_data, test_data = output.randomSplit([0.7,0.3])

In [0]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(labelCol= 'crew')
lr_model = lr.fit(train_data)
test_results = lr_model.evaluate(test_data)
test_results.residuals.show()    # difference ground truth vs prediction
print('RMSE:',test_results.rootMeanSquaredError)    # RMSE gives the average difference (compare to min)
print('r2:',test_results.r2)    # accuracy

+--------------------+
|           residuals|
+--------------------+
| -1.2513921382307167|
|-0.03005580976992306|
| -0.5729568912117067|
| 0.19150012383494164|
| -0.2681595345210379|
|  0.7505582225026339|
|  0.8598792707149965|
|  0.0527583450559268|
|  0.9414116444300227|
| -0.8873968176710232|
|  0.5824758937846575|
|  0.2928655533936002|
|   0.825042332123143|
| -1.0861603303549217|
| -1.3229099880678454|
|-0.24978210470074913|
|  0.6543639633557792|
| 0.43876222163272516|
| -1.6778840656271239|
| 0.14790619831827723|
+--------------------+
only showing top 20 rows

RMSE: 0.7719295588562446
r2: 0.9295981954963772


In [0]:
unlabeled_data = test_data.select('features')
predictions = lr_model.transform(unlabeled_data)
predictions.show()

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|[12.0,138.0,31.14...| 13.101392138230716|
|[21.0,28.43,8.08,...|  4.030055809769923|
|[9.0,85.0,19.68,9...|  9.262956891211706|
|[22.0,3.341,0.66,...|0.39849987616505833|
|[9.0,116.0,26.0,9...| 11.268159534521038|
|[18.0,70.60600000...|  7.829441777497366|
|[11.0,91.0,20.32,...|  9.130120729285004|
|[25.0,34.25,10.52...|  4.647241654944073|
|[11.0,90.0,22.4,9...| 10.058588355569977|
|[17.0,101.353,26....| 10.887396817671023|
|[22.0,70.367,20.5...|  8.617524106215342|
|[6.0,113.0,37.82,...|   11.7071344466064|
|[23.0,48.563,20.2...|  5.884957667876857|
|[16.0,74.137,19.5...|  8.686160330354921|
|[13.0,138.0,31.14...| 13.082909988067845|
|[7.0,158.0,43.7,1...| 13.849782104700749|
|[17.0,77.71300000...|   8.43563603664422|
|[28.0,46.052,14.5...| 6.1612377783672745|
|[5.0,160.0,36.34,...| 15.277884065627124|
|[15.0,30.27699999...| 3.8520938016817228|
+----------

In [0]:
from pyspark.sql.functions import corr
data.select(corr('crew','passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+

